In [1]:
import os
import numpy as np
import rasterio
import matplotlib.image 


In [2]:
files = {}
for f in os.listdir("data"):
    if ".jp2" in f:
        parts = f.split("_")
        name = "_".join(parts[:2])
        if name not in files:
            files[name] = {}
        path = f"data/{f}"
        files[name][parts[2]] = path#, os.path.getmtime(path)]


In [3]:
files = {k: v for k,v in files.items() if len(v) == 5 and not (os.path.isfile(f"images/{k}.jpeg") and os.path.isfile(f"forests/{k}.jpeg"))}


In [4]:
files


{'T35VMD_20150804T094006': {'B02': 'data/T35VMD_20150804T094006_B02_10m.jp2',
  'B03': 'data/T35VMD_20150804T094006_B03_10m.jp2',
  'B04': 'data/T35VMD_20150804T094006_B04_10m.jp2',
  'B08': 'data/T35VMD_20150804T094006_B08_10m.jp2',
  'SCL': 'data/T35VMD_20150804T094006_SCL_20m.jp2'},
 'T35VMC_20151003T094016': {'B02': 'data/T35VMC_20151003T094016_B02_10m.jp2',
  'B03': 'data/T35VMC_20151003T094016_B03_10m.jp2',
  'B04': 'data/T35VMC_20151003T094016_B04_10m.jp2',
  'B08': 'data/T35VMC_20151003T094016_B08_10m.jp2',
  'SCL': 'data/T35VMC_20151003T094016_SCL_20m.jp2'},
 'T34UEG_20150820T100016': {'B02': 'data/T34UEG_20150820T100016_B02_10m.jp2',
  'B03': 'data/T34UEG_20150820T100016_B03_10m.jp2',
  'B04': 'data/T34UEG_20150820T100016_B04_10m.jp2',
  'B08': 'data/T34UEG_20150820T100016_B08_10m.jp2',
  'SCL': 'data/T34UEG_20150820T100016_SCL_20m.jp2'},
 'T34UFG_20150807T095006': {'B02': 'data/T34UFG_20150807T095006_B02_10m.jp2',
  'B03': 'data/T34UFG_20150807T095006_B03_10m.jp2',
  'B04': 

In [5]:
len(files)


72

In [ ]:
for k,v in files.items():
    try:
        red = rasterio.open(v["B04"], driver="JP2OpenJPEG").read(1)
        green = rasterio.open(v["B03"], driver="JP2OpenJPEG").read(1)
        blue = rasterio.open(v["B02"], driver="JP2OpenJPEG").read(1)

        gain = 2
        red_n = np.clip(red * gain / 10000, 0, 1)
        green_n = np.clip(green * gain / 10000, 0, 1)
        blue_n = np.clip(blue * gain / 10000, 0, 1)
        rgb_composite_n = np.dstack((red_n, green_n, blue_n))
        #plt.imshow(rgb_composite_n)
        outfile = f"images/{k}.jpeg"
        matplotlib.image.imsave(outfile, rgb_composite_n)
        print("Saved as:", outfile)

        b04 = rasterio.open(v["B04"], driver="JP2OpenJPEG").read(1)/10000
        b08 = rasterio.open(v["B08"], driver="JP2OpenJPEG").read(1)/10000
        ndvi = ((b08-b04)/(b08+b04))
        # Create composite image
        clipped = np.clip(ndvi, 0.0, 1.0)
        clipped[clipped < 0.6] = 0.0
        rgb_composite_n = np.dstack((np.zeros(ndvi.shape), clipped, np.zeros(ndvi.shape)))
        outfile = f"forests/{k}.jpeg"
        matplotlib.image.imsave(outfile, rgb_composite_n)
        print("Saved as:", outfile)
    except BaseException as e:
        print(f"ERROR: {k} failed with {e}")

Saved as: images/T35VMD_20150804T094006.jpeg
ERROR: T35VMD_20150804T094006 failed with 'data/T35VMD_20150804T094006_B08_10m.jp2' not recognized as being in a supported file format.
ERROR: T35VMC_20151003T094016 failed with 'data/T35VMC_20151003T094016_B04_10m.jp2' not recognized as being in a supported file format.
Saved as: images/T34UEG_20150820T100016.jpeg
ERROR: T34UEG_20150820T100016 failed with 'data/T34UEG_20150820T100016_B08_10m.jp2' not recognized as being in a supported file format.
ERROR: T34UFG_20150807T095006 failed with 'data/T34UFG_20150807T095006_B04_10m.jp2' not recognized as being in a supported file format.
Saved as: images/T34VFJ_20151208T100412.jpeg


/tmp/ipykernel_123118/3446437123.py:19: RuntimeWarning: invalid value encountered in divide
  ndvi = ((b08-b04)/(b08+b04))


Saved as: forests/T34VFJ_20151208T100412.jpeg
Saved as: images/T35UMB_20150808T092006.jpeg
Saved as: forests/T35UMB_20150808T092006.jpeg
Saved as: images/T34UDG_20151006T095016.jpeg
Saved as: forests/T34UDG_20151006T095016.jpeg
Saved as: images/T35UNB_20150910T092726.jpeg
Saved as: forests/T35UNB_20150910T092726.jpeg
Saved as: images/T35VNE_20150804T094006.jpeg
Saved as: forests/T35VNE_20150804T094006.jpeg
